In [1]:
import numpy as np
import pandas as pd
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

## Acquire Data

In [2]:
sf_df = pd.read_csv('./input/existing-commercial-wireless-telecommunication-services-facilities-in-san-francisco.csv')

## Analyze Data

In [3]:
# sf_df.head(3)
sf_df.columns
# sf_df.tail()
# sf_df.columns
# sf_df['Location']

Index(['Carrier', 'ID ', 'Search Ring Name ID', 'Block', 'Lot',
       'Zoning District', 'Hight and Bulk District', 'Type of Building',
       'Preference (1-7)', 'Type of Consumer Services',
       'Type of Consumer Products', 'Number of Antennas', 'Location of BTS',
       'Type of Technology', 'Radio Frequency Range (Megahertz)',
       'Power Output of Equipment (Watts)', 'Height to Top of Antenna',
       'Name of Community Liason', 'Phone Number of Community Liason',
       'Comments', 'Location 1'],
      dtype='object')

In [4]:
sf_df.columns
service = sf_df['Location of BTS']

dic = {}
for data in service:
    if data not in dic:
        dic[data] = 1
    else:
        dic[data] += 1

print (list(dic.keys()))

['Basement', 'Ground Fl ', 'Roof', 'Interior', 'Lower Roof', 'oDAS', 'basement', '6th fl ', 'Basement.', '2nd fl', 'Garage', '2nd Fl, telco rm', 'First fl', '2nd fl balcony -lower roof', 'Roof ', 'Basement & Roof', nan, 'Equip Rm', 'Lower Garage', '6th fl', 'Central Office', 'Pole', 'Penthouse Interior', 'Ground Floor', 'Tenant Improvement', 'Ground', 'In parking garage', 'In bldg on 4th floor', 'Parking Garage', 'Building', 'TBD', 'Indoor', 'Ground, Outdoor', '2nd Level', 'Bell Tower', 'pole', 'Internal', 'Penthouse', 'Shelter', 'Ground ', 'Rooftop', 'Outdoors', 'Steeple', 'indoor', 'roof', 'Indoor ', 'Third Level', 'Ground Level', 'Inside Penthouse', 'Lower roof', 'Rooftop Penthouse', 'Garage Roof']


## Generate New Column

In [5]:
Basement = ['Basement', 'basement', 'Basement.', 'Basement & Roof']
Low = ['Ground Fl ', '2nd fl', 'First fl', '2nd Fl, telco rm', 
        'Ground Floor', 'Ground','Ground, Outdoor', 
       '2nd Level', 'Ground ', 'Ground Level']

Indoor = Basement + Low + ['Interior', '6th fl ', 'Equip Rm', 'Central Office','Penthouse Interior', 
                           'Tenant Improvement', 'In parking garage', 'In bldg on 4th floor', 'Parking Garage', 'Building',
                          'Indoor', 'Internal', 'Penthouse', 'Shelter', 'indoor', 'Indoor ', 'Third Level', 'Inside Penthouse']

Top = ['Roof', 'Lower Roof', '2nd fl balcony -lower roof', 'Basement & Roof','Pole', 'Bell Tower', 'pole', 
       'Rooftop', 'Steeple', 'roof', 'Lower roof','Rooftop Penthouse', 'Garage Roof']

Outside = ['oDAS', 'TBD', 'Outdoors']



In [6]:
ind = []
top = []
out = []
loc = sf_df['Location of BTS']
for i in range(loc.size):
    if str(loc[i]) in Indoor:
        ind.append(1.0)
    else:
        ind.append(0.0)
    if str(loc[i]) in Top:
        top.append(1.0)
    else:
        top.append(0.0)
    if str(loc[i]) in Outside:
        out.append(1.0)
    else:
        out.append(0.0)

pdind = pd.Series(ind)
pdtop = pd.Series(top)
pdout = pd.Series(out)

sf_df['INDBTS'] = pdind
sf_df['TOPBTS'] = pdtop
sf_df['OUTBTS'] = pdout

In [7]:
# sf_df.to_csv('./temp1.csv', index=False)

In [8]:
sf_df.columns

Index(['Carrier', 'ID ', 'Search Ring Name ID', 'Block', 'Lot',
       'Zoning District', 'Hight and Bulk District', 'Type of Building',
       'Preference (1-7)', 'Type of Consumer Services',
       'Type of Consumer Products', 'Number of Antennas', 'Location of BTS',
       'Type of Technology', 'Radio Frequency Range (Megahertz)',
       'Power Output of Equipment (Watts)', 'Height to Top of Antenna',
       'Name of Community Liason', 'Phone Number of Community Liason',
       'Comments', 'Location 1', 'INDBTS', 'TOPBTS', 'OUTBTS'],
      dtype='object')

In [9]:
service = sf_df['Type of Consumer Services']

dic = {}
for data in service:
    if data not in dic:
        dic[data] = 1
    else:
        dic[data] += 1

print (dic)

{'voice, video & data': 259, 'Voice, Data, High Speed Data': 313, 'Voice/Data, High Speed Data': 1, 'Voice/Data': 12, 'Voice, High Speed Data': 220, 'Wireless Broadband Internet, Voice & Data Transmissions': 23, 'voice, video, data': 316, nan: 1, 'Voice & Data Transmissions': 108}


In [10]:
temp = pd.read_csv('./input/result.csv')

In [11]:
temp.columns

Index(['Unnamed: 0', 'Carrier', 'voice', 'data', 'high speed', 'wireless',
       'video', 'Comercial', 'Residencial', 'Public', 'Office', 'Industrial',
       'General', 'PCS', 'AWS', 'GHz', '1900', '2100', '700', '2496', 'INDBTS',
       'TOPBTS', 'OUTBTS'],
      dtype='object')

## Model and Train 

In [12]:
solution = pd.read_csv('./input/result.csv')

In [13]:
solution.drop(['Unnamed: 0'], axis=1, inplace=True)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(solution.iloc[:, 1:], solution.iloc[:, 0], test_size=0.2, random_state=42)

In [15]:
Y_test.describe()

count    251.000000
mean       0.219124
std        0.414479
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: Carrier, dtype: float64

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [17]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

100.0

In [18]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

100.0

In [19]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

100.0

In [20]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

seed = 7
np.random.seed(seed)

mymodel = Sequential()

mymodel.add(Dense(25, activation='relu', input_dim = 21))
mymodel.add(Dropout(0.5))

mymodel.add(Dense(1, activation='sigmoid'))

optimizer = SGD(lr = 0.01, momentum=0.9)

mymodel.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

mymodel.fit(X_train, Y_train, batch_size=10, epochs=50)

y_pred = mymodel.predict(X_test)
y_pred = np.round_(y_pred, 0)

Using TensorFlow backend.


Epoch 1/50
1002/1002 [==============================] - 1s 674us/step - loss: 0.3653 - acc: 0.8503
Epoch 2/50
1002/1002 [==============================] - 0s 151us/step - loss: 0.0780 - acc: 0.9860
Epoch 3/50
1002/1002 [==============================] - 0s 155us/step - loss: 0.0432 - acc: 0.9930
Epoch 4/50
1002/1002 [==============================] - 0s 152us/step - loss: 0.0297 - acc: 0.9930
Epoch 5/50
1002/1002 [==============================] - 0s 158us/step - loss: 0.0430 - acc: 0.9940
Epoch 6/50
1002/1002 [==============================] - 0s 169us/step - loss: 0.0195 - acc: 0.9980
Epoch 7/50
1002/1002 [==============================] - 0s 167us/step - loss: 0.0166 - acc: 0.9980
Epoch 8/50
1002/1002 [==============================] - 0s 151us/step - loss: 0.0123 - acc: 0.9980
Epoch 9/50
1002/1002 [==============================] - 0s 157us/step - loss: 0.0172 - acc: 0.9970
Epoch 10/50
1002/1002 [==============================] - 0s 159us/step - loss: 0.0118 - acc: 0.9990
Epoch 11/